In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [20]:
train_path = 'model_train'
valid_path = 'model_valid'
test_path = 'model_test'

In [21]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), class_mode='binary', batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), class_mode='binary', batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224),  class_mode='binary', batch_size=10, shuffle=False)

Found 999 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [22]:
vgg16_model = tf.keras.applications.vgg16.VGG16()

model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [23]:
for layer in model.layers:
    layer.trainable = False
    
model.add(Dense(units=1, activation = 'sigmoid'))

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [25]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='BinaryCrossentropy', metrics=['accuracy'])
model.fit(x=train_batches,
          steps_per_epoch=len(train_batches),
          validation_data=valid_batches,
          validation_steps=len(valid_batches),
          epochs=5,
          verbose=2
)

Epoch 1/5
100/100 - 280s - loss: 0.4374 - accuracy: 0.8068 - val_loss: 0.2086 - val_accuracy: 0.9500
Epoch 2/5
100/100 - 275s - loss: 0.1135 - accuracy: 0.9760 - val_loss: 0.1066 - val_accuracy: 0.9700
Epoch 3/5
100/100 - 288s - loss: 0.0734 - accuracy: 0.9820 - val_loss: 0.0706 - val_accuracy: 0.9800
Epoch 4/5
100/100 - 309s - loss: 0.0553 - accuracy: 0.9870 - val_loss: 0.0573 - val_accuracy: 0.9800
Epoch 5/5
100/100 - 286s - loss: 0.0449 - accuracy: 0.9880 - val_loss: 0.0444 - val_accuracy: 1.0000


In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=1)
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.round(predictions))
print(cm)
accuracy_score(y_true=test_batches.classes, y_pred=np.round(predictions))

20/20 [==============================] - 48s 2s/step
[[100   0]
 [  7  93]]


0.965

In [27]:
model.save('MaskDetectionTrained.h5')